In [3]:
import requests
import os

def url_to_file(url):
    with open(os.path.join(folder_name,
                           url.split('/')[-1]), mode = 'wb') as file:
        file.write(requests.get(url).content)

In [11]:
# Make directory if it doesn't already exist
folder_name = 'project_data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# check for twitter_archive_enhanced.csv
file_path = os.path.join(folder_name, 'twitter_archive_enhanced.csv')
assert os.path.isfile(file_path), "Download twitter_archive_enhanced.csv"

# check for image-predictions.tsv
# if doesn't exist, download programatically
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file_path = os.path.join(folder_name, url.split('/')[-1])
if not os.path.isfile(file_path):
    with open(file_path, mode = 'wb') as file:
        file.write(requests.get(url).content)
        


AssertionError: Download twitter_archive_enhanced.csv

In [1]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file_name = url.split('/')
file_name[-1]

'image-predictions.tsv'

In [ ]:
# Your code here
# Implement the code in the video above in a for loop for all Ebert reviews
def url_to_file(url):
    with open(os.path.join(folder_name,
                           url.split('/')[-1]), mode = 'wb') as file:
        file.write(requests.get(url).content)

for file_url in ebert_review_urls:
    url_to_file(file_url)

In [ ]:
r = requests.get(first_image_url)